In [ ]:
#reference pytorch

np.arange(3.0)

x = torch.empty(5,3) # will have some junk
print(x[0,0])

torch.zeros_like(x)
torch.zeros(5,3)

y = torch.rand(5,3)
print(y)

x.size()

torch.tensor([2,1,4])

torch.add(x,y)

z = torch.tensor([1.234])
z.item() #will work for one item to extract out

In [ ]:
"""
NEVER RUN THIS
"""

# input data 1
assert(1==0)

path = '11_ds_1/'
class_names = os.listdir(path)
print(class_names)
data = []
target = []

for cn in range(len(class_names)):
    for fname in os.listdir(path+class_names[cn]+'/'):
        data.append(np.loadtxt(path+class_names[cn]+'/'+fname).flatten())
        target.append(int(cn)) 
print(np.shape(data))
        
# test train split

train_data, test_data, train_target, test_target = train_test_split(data,target,test_size=0.2)
np.save(path+'class_names.npy', class_names)
np.save(path+'train_data.npy', train_data)
np.save(path+'test_data.npy', test_data)
np.save(path+'test_target.npy', test_target)
np.save(path+'train_target.npy', train_target)
print('saved')

In [ ]:
"""
NEVER RUN THIS
"""

# input data 2
assert(1==0)

path = '11_ds_2/'
class_names = os.listdir(path)
print(class_names)
data = []
target = []
for cn in range(len(class_names)):
    data.append(np.genfromtxt(path+class_names[cn], delimiter=','))
s = np.shape(data)
print(s)
for i in range(s[0]):
    for j in range(s[1]):
        target.append(i)
data = np.reshape(data,(s[0]*s[1],s[2]))
print(np.shape(data),np.shape(target))

# test train split

train_data, test_data, train_target, test_target = train_test_split(data,target,test_size=0.2)
np.save(path+'class_names.npy', class_names)
np.save(path+'train_data.npy', train_data)
np.save(path+'test_data.npy', test_data)
np.save(path+'test_target.npy', test_target)
np.save(path+'train_target.npy', train_target)
print('saved')

In [1]:
# imports

import numpy as np
import torch
import os
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from sklearn import decomposition
import torch.optim as optim

In [2]:
# task1

    Notes
    1. Took whole file as single dim for ds1

In [29]:
dir_path = '11_ds_1/'
class_names = np.load(dir_path+'class_names.npy')
train_data = np.load(dir_path+'train_data.npy')
train_target = np.load(dir_path+'train_target.npy')
test_data = np.load(dir_path+'test_data.npy')
test_target = np.load(dir_path+'test_target.npy')
print(np.shape(train_data),np.shape(test_data))
print(class_names)

(1329, 828) (333, 828)
['coast' 'forest' 'insidecity' 'mountain' 'street']


In [30]:
# parameters

dim = 500
h1_dim = 50
out_dim = len(class_names)
learning_rate = 0.001
num_epochs = 600
momentum = 0.9
batch_size = 32  # general rule is to try 32, 64, 128 ...

In [5]:
# PCA

pca = decomposition.PCA(n_components = 500)
pca.fit(train_data)
print(np.shape(train_data))
train_data_pca = pca.transform(train_data)
test_data_pca = pca.transform(test_data)
print(np.shape(train_data))

(1329, 828)
(1329, 500)


In [10]:
# MLFFNN model

class MLFFNN(nn.Module):
    
    def __init__(self):
        super(MLFFNN, self).__init__()
        self.fc1 = nn.Linear(dim,h1_dim) # linear -> fully connected layers
        self.fc2 = nn.Linear(h1_dim,out_dim)
        
    def forward(self, x):
        x = torch.sigmoid(self.fc1(x))
        x = self.fc2(x)
        return x

In [14]:
# loss fun and update method

tensor_train_data = torch.Tensor(train_data_pca)
tensor_train_target = torch.LongTensor(train_target) # long otherwise takes as float
 
tensor_test_data = torch.Tensor(test_data_pca)
tensor_test_target = torch.LongTensor(test_target)

# num_workers - how many parallel data loadings
train_dataset = torch.utils.data.TensorDataset(tensor_train_data,tensor_train_target)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size,shuffle=True,num_workers=8)

test_dataset = torch.utils.data.TensorDataset(tensor_test_data,tensor_test_target)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size,shuffle=False,num_workers=8)

net = MLFFNN()
criterion = nn.CrossEntropyLoss()
# ASDG for generalised delta ig
# schochastic gradient descent with nestrov momentum
optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=momentum)
# optimizer = optim.Adagrad(net.parameters(), lr=learning_rate,lr_decay=0)

In [15]:
# training


running_loss = 0.0
count = 0
for epoch in range(num_epochs):
    
    for i, load_data in enumerate(train_loader):
        
        inputs, labels = load_data
        
        # initialize with zeros all param
        optimizer.zero_grad()
        
        outputs = net(inputs)
        loss = criterion(outputs,labels)
        loss.backward()
        optimizer.step()
        count += 1
        running_loss += loss.item()
        
    if((epoch+1)%50 == 0):
        print(epoch+1, running_loss/count)
        running_loss = 0.0
        count = 0

50 1.107709253515516
100 0.6830926092962424
150 0.5093000791825
200 0.41182342586063203
250 0.3457612983350243
300 0.2953531828345287
350 0.2538541241415909
400 0.21813349328225567
450 0.1873619727063037
500 0.16065591363679796
550 0.13775483613035508
600 0.11814552073677381


In [ ]:
# retrive saved net

net = MLFFNN()
net.load_state_dict(torch.load('net1_task1.pth'))

In [17]:
# testing

# train data
correct = 0
total = 0
with torch.no_grad():
    for data in train_loader:
        inputs, labels = data
        outputs = net(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print('train accuracy',end=':')
print(correct/total)

#test data
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        inputs, labels = data
        outputs = net(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print('test accuracy',end=':')
print(correct/total)

train accuracy:0.9932279909706546
test accuracy:0.7027027027027027


In [18]:
# save 
# model1 is saved after fine tuning

saved_net_path = 'net1_task1.pth'
torch.save(net.state_dict(),saved_net_path)

In [31]:
dim = 828
h1_dim = 1000
h2_dim = 800
h3_dim = 500
h4_dim = 900
batch_size = 64
learning_rate = 0.001
num_epochs = 800
num_workers = 8

In [41]:
# AANN

class AANN(nn.Module):
    def __init__(self):
        super(AANN, self).__init__()
        self.fc1 = nn.Linear(dim,h1_dim)
        self.fc2 = nn.Linear(h1_dim,h2_dim)
        self.fc5 = nn.Linear(h2_dim, h3_dim)
        self.fc3 = nn.Linear(h3_dim,h4_dim)
        self.fc4 = nn.Linear(h4_dim,dim)
        
    def forward(self, x):
        m = nn.Tanh()
        x = self.fc1(x)
        x = m(self.fc2(x))
        y = self.fc5(x)
        x = m(self.fc3(y))
        x = self.fc4(x)
        return x, y

In [46]:
# training for AANN

tensor_data = torch.Tensor(train_data)
tensor_data_test = torch.Tensor(test_data)

dataset = torch.utils.data.TensorDataset(tensor_data,tensor_data)
dataset_test = torch.utils.data.TensorDataset(tensor_data_test,tensor_data_test)
loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,shuffle=True,num_workers=num_workers)
loader_test = torch.utils.data.DataLoader(dataset_test, batch_size=batch_size,shuffle=True,num_workers=num_workers)

aann = AANN()
criterion = nn.MSELoss(reduction='mean')
optimizer = optim.Adagrad(aann.parameters(), lr=learning_rate,lr_decay=0)

In [47]:
running_loss = 0.0
count = 0
for epoch in range(num_epochs):
    
    for i, load_data in enumerate(loader):
        
        inputs, same_inputs = load_data
        
        # initialize with zeros all param
        optimizer.zero_grad()
        
        outputs,_ = aann(inputs)
        loss = criterion(outputs,same_inputs)
        loss.backward()
        optimizer.step()
        count += 1
        running_loss += loss.item()
        
    if((epoch+1)%50 == 0):
        print(epoch+1, running_loss/count)
        running_loss = 0.0
        count = 0

50 0.15177173100057104
100 0.09829308004606338
150 0.07371671600001199
200 0.05912337257038979
250 0.049544438833282106
300 0.04272164853201026
350 0.03761483048754079
400 0.033675467130683714
450 0.030534732834923833
500 0.027969556527123563


In [ ]:
aann = AANN()
net.load_state_dict(torch.load('aann_task1.pth'))

In [48]:
error = 0
num = 0
with torch.no_grad():
    for data in loader:
        inputs, labels = data
        outputs,_ = aann(inputs)
        diff = outputs-inputs
        error += ((diff*diff).sum().item())
        num += 1
print('Train error',end=':')
print(error/(num*dim))

error = 0
num = 0
with torch.no_grad():
    for data in loader_test:
        inputs, labels = data
        outputs,_ = aann(inputs)
        diff = outputs-inputs
        error += ((diff*diff).sum().item())
        num += 1
print('Test error',end=':')
print(error/(num*dim))

Train error:1.6991542116757066
Test error:2.223556340412625


In [49]:
torch.save(aann.state_dict(),'aann_task1.pth')

In [50]:
aann_train = torch.Tensor(train_data)
aann_test = torch.Tensor(test_data)

In [51]:
with torch.no_grad():
    print(aann_train.size())
    _, aann_train = aann(aann_train)
    _, aann_test = aann(aann_test)
    print(aann_train.size())

torch.Size([1329, 828])
torch.Size([1329, 500])


In [ ]:
# parameters

dim = 500
h1_dim = 50
out_dim = len(class_names)
learning_rate = 0.001
momentum = 0.9
num_workers = 8
num_epochs = 1000
batch_size = 32  # general rule is to try 32, 64, 128 ...

In [ ]:
# loss fun and update method

tensor_train_target = torch.LongTensor(train_target) # long otherwise takes as float

tensor_test_target = torch.LongTensor(test_target)

# num_workers - how many parallel data loadings
train_dataset = torch.utils.data.TensorDataset(aann_train,tensor_train_target)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size,shuffle=True,num_workers=num_workers)

test_dataset = torch.utils.data.TensorDataset(aann_test,tensor_test_target)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size,shuffle=False,num_workers=num_workers)

net2 = MLFFNN()
criterion = nn.CrossEntropyLoss()
# ASDG for generalised delta ig
# schochastic gradient descent with nestrov momentum
optimizer = optim.SGD(net2.parameters(), lr=learning_rate, momentum=momentum)
# optimizer = optim.Adagrad(net.parameters(), lr=learning_rate,lr_decay=0)

In [ ]:
# training


running_loss = 0.0
count = 0
for epoch in range(num_epochs):
    
    for i, load_data in enumerate(train_loader):
        
        inputs, labels = load_data
        
        # initialize with zeros all param
        optimizer.zero_grad()
        
        outputs = net2(inputs)
        loss = criterion(outputs,labels)
        loss.backward()
        optimizer.step()
        count += 1
        running_loss += loss.item()
        
    if((epoch+1)%50 == 0):
        print(epoch+1, running_loss/count)
        running_loss = 0.0
        count = 0

In [ ]:
net2 = MLFFNN()
net2.load_state_dict(torch.load('net2_task1.pth'))

In [ ]:
# testing

# train data
correct = 0
total = 0
with torch.no_grad():
    for data in train_loader:
        inputs, labels = data
        outputs = net2(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print('train accuracy',end=':')
print(correct/total)

#test data
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        inputs, labels = data
        outputs = net2(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print('test accuracy',end=':')
print(correct/total)

In [ ]:
torch.save(net2.state_dict(),'net2_task1.pth')